<a href="https://colab.research.google.com/github/derektorquette/pln-com-deep-learning-ia-expert/blob/main/tradu%C3%A7%C3%A3o_de_en_para_pt_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etapa 1: Importação das bibliotecas

In [1]:
import numpy as np
import math
import re
import time
import zipfile
import random
from google.colab import drive
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Etapa 2: Pré-processamento dos dados

## Carregamento da base de dados

- Bases de dados: https://www.statmt.org/europarl/

In [2]:
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/CURSOS LIVRES FORMAÇÃO COMPLEMENTAR/3. Udemy/Processamento de Linguagem Natural com Deep Learning/Seção 4 e 5 - Arquitetura Tranformer /pt-en.zip'
zip_object = zipfile.ZipFile(file = path, mode = 'r')
zip_object.extractall('./')
zip_object.close()

In [4]:
with open('/content/pt-en/europarl-v7.pt-en.en', mode='r', encoding='utf-8') as f:
  europarl_en = f.read()
with open('/content/pt-en/europarl-v7.pt-en.pt', mode='r', encoding='utf-8') as f:
  europarl_pt = f.read ()

In [6]:
europarl_en[0:100]

'Resumption of the session\nI declare resumed the session of the European Parliament adjourned on Frid'

In [7]:
en = europarl_en.split('\n')

In [8]:
len(en) # total de frases

1960408

In [ ]:
en

In [11]:
pt = europarl_pt.split('\n')

In [12]:
len(pt)

1960408

In [13]:
for _ in range(5):
  print('-----')
  i = random.randint(0, len(en) - 1)
  print(en[i])
  print(pt[i])

-----
Let us never forget how much the European Union is already doing and how great is its contribution to greater fairness for them.
Não esqueçamos nunca tudo o que a União Europeia já estar a fazer, nem a importância do seu contributo para que estes países gozem de uma maior equidade.
-----
On 2 June 2006 – just two weeks ago – the Commission submitted to the Tunisian authorities the draft internal regulation of the abovementioned subcommittee and is awaiting a reply from the Tunisian side.
Em 2 e Junho de 2006 – há apenas duas semanas – a Comissão apresentou às autoridades tunisinas o projecto de regulamento interno da referida subcomissão e aguarda a resposta da parte tunisina.
-----
The area is currently governed by national laws, and citizens are poorly informed with regard to the options, reimbursement and possible treatment abroad.
Actualmente, esta matéria é regida apenas pelas legislações nacionais, estando os cidadãos pouco informados sobre as opções à sua disposição, sobre

## Limpeza dos dados

In [14]:
corpus_en = europarl_en
corpus_en = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", corpus_en)
corpus_en = re.sub(r".\$\$\$", '', corpus_en)
corpus_en = re.sub(r" +", " ", corpus_en)
corpus_en = corpus_en.split('\n')

In [15]:
corpus_pt = europarl_pt
corpus_pt = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", corpus_pt)
corpus_pt = re.sub(r".\$\$\$", '', corpus_pt)
corpus_pt = re.sub(r" +", " ", corpus_pt)
corpus_pt = corpus_pt.split('\n')

In [16]:
len(corpus_en)

1960408

In [17]:
len(corpus_pt)

1960408


## Tokenização

In [18]:
2**13

8192

In [ ]:
# Atualização MAR-2024
tokenizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(corpus_en, target_vocab_size=2**13)

In [ ]:
tokenizer_en.vocab_size

8191

In [ ]:
# Atualização MAR-2024
tokenizer_pt = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(corpus_pt, target_vocab_size=2**13)

In [ ]:
tokenizer_pt.vocab_size

8116

In [ ]:
vocab_size_en = tokenizer_en.vocab_size + 2
vocab_size_pt = tokenizer_pt.vocab_size + 2

In [ ]:
inputs = [[vocab_size_en - 2] + tokenizer_en.encode(sentence) + [vocab_size_en - 1] for sentence in corpus_en]

In [ ]:
for _ in range(5):
  print(inputs[random.randint(0, len(inputs) - 1)])

[8191, 25, 6690, 7967, 9, 2904, 6113, 2, 5, 20, 9, 19, 445, 4, 2120, 7981, 8192]
[8191, 25, 179, 9, 1223, 7981, 8192]
[8191, 47, 9, 7, 2161, 2, 2383, 3192, 7981, 8192]
[8191, 2963, 64, 2900, 8, 20, 9, 77, 27, 1437, 553, 24, 2651, 639, 2, 1093, 2, 973, 5, 3402, 224, 367, 112, 517, 1567, 224, 8, 64, 26, 1512, 4, 1, 310, 3, 23, 913, 7998, 8192]
[8191, 7048, 2, 11, 137, 1, 71, 35, 1283, 8, 29, 7755, 801, 12, 35, 96, 1447, 261, 12, 264, 3, 1, 255, 138, 2, 5, 690, 16, 161, 6, 1, 329, 10, 5160, 2869, 7981, 8192]


In [ ]:
outputs = [[vocab_size_pt - 2] + tokenizer_pt.encode(sentence) + [vocab_size_pt - 1] for sentence in corpus_pt]

In [ ]:
for _ in range(5):
  print(outputs[random.randint(0, len(outputs) - 1)])

[8116, 322, 3111, 1, 2626, 279, 1763, 13, 598, 33, 3, 2738, 2065, 1, 4, 61, 47, 106, 1851, 7968, 5521, 3, 185, 166, 54, 7905, 94, 5, 4, 61, 37, 106, 1851, 7968, 5521, 3, 20, 283, 7, 957, 8, 1442, 4378, 1975, 1, 10, 7254, 1154, 6801, 7957, 1, 328, 2707, 347, 3, 4775, 454, 7906, 8117]
[8116, 6871, 201, 4, 3, 7807, 152, 4168, 5, 2660, 377, 4732, 7977, 7906, 8117]
[8116, 545, 403, 197, 83, 24, 2774, 8, 1056, 15, 5579, 15, 3302, 4663, 7892, 28, 5925, 135, 1, 1056, 160, 4, 2611, 202, 1, 1253, 1, 919, 4326, 11, 4806, 1687, 29, 83, 6, 672, 8, 1928, 7, 298, 6744, 7906, 8117]
[8116, 507, 18, 3, 60, 2, 1701, 8, 1741, 7906, 8117]
[8116, 36, 1155, 997, 7, 1703, 58, 6, 2779, 5, 6, 2956, 109, 37, 3878, 29, 5, 277, 228, 3259, 14, 504, 284, 5, 28, 5342, 42, 6, 363, 328, 228, 76, 335, 1165, 5, 14, 741, 6415, 11, 59, 470, 7906, 8117]


## Remoção de sentenças muito longas

In [ ]:
max_length = 15
idx_to_remove = [count for count, sent in enumerate(inputs) if len(sent) > max_length]

In [ ]:
len(idx_to_remove)

1685300

0 - 1
1 - 13
2 - 15

In [ ]:
for idx in reversed(idx_to_remove):
  del inputs[idx]
  del outputs[idx]

In [ ]:
idx_to_remove = [count for count, sent in enumerate(outputs) if len(sent) > max_length]

In [ ]:
len(idx_to_remove)

66118

In [ ]:
for idx in reversed(idx_to_remove):
  del inputs[idx]
  del outputs[idx]

In [ ]:
len(inputs)

208990

In [ ]:
len(outputs)

208990

## Padding e batches

In [ ]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, value=0, padding = 'post', maxlen=max_length)
outputs = tf.keras.preprocessing.sequence.pad_sequences(outputs, value=0, padding = 'post', maxlen=max_length)

In [ ]:
for _ in range(5):
  print(outputs[random.randint(0, len(outputs) - 1)])

[8116 1438 5889 7657 1075   19    2  365   49 4393 7906 8117    0    0
    0]
[8116 3562 1111    1  700    1 1051    6  761 7906 8117    0    0    0
    0]
[8116   30  410  408  363 1405 7906 8117    0    0    0    0    0    0
    0]
[8116   90  109  959 7893 8117    0    0    0    0    0    0    0    0
    0]
[8116 1984    6  898    1   17  744 1003 3977 7906 8117    0    0    0
    0]


from_tensor_slices: https://www.geeksforgeeks.org/tensorflow-tf-data-dataset-from_tensor_slices/

cache e prefetch: https://www.tensorflow.org/guide/data_performance

In [ ]:
batch_size = 64
buffer_size = 20000

dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))
dataset = dataset.cache()
dataset = dataset.shuffle(buffer_size).batch(batch_size)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

# Etapa 3: Construção do modelo

## Embedding

Positional encoding:

$PE_{(pos,2i)} =\sin(pos/10000^{2i/dmodel})$

$PE_{(pos,2i+1)} =\cos(pos/10000^{2i/dmodel})$

- The positional encodings have the same dimension dmodel
as the embeddings, so that the two can be summed

In [ ]:
class PositionalEncoding(layers.Layer):

    def __init__(self):
      super(PositionalEncoding, self).__init__()

    def get_angles(self, pos, i, d_model):
      angles = 1 / np.power(10000., (2*(i // 2)) / np.float32(d_model))
      return pos * angles # (seq_lenght, d_model)

    def call(self, inputs):
      seq_lenght = inputs.shape.as_list()[-2]
      d_model = inputs.shape.as_list()[-1]
      angles = self.get_angles(np.arange(seq_lenght)[:, np.newaxis],
                               np.arange(d_model)[np.newaxis, :], d_model)
      angles[:, 0::2] = np.sin(angles[:, 0::2])
      angles[:, 1::2] = np.cos(angles[:, 1::2])
      pos_encoding = angles[np.newaxis, ...]
      return inputs + tf.cast(pos_encoding, tf.float32)

## Mecanismo de atenção

### Cálculo da atenção

$Attention(Q, K, V ) = \text{softmax}\left(\dfrac{QK^T}{\sqrt{d_k}}\right)V $

In [ ]:
def scaled_dot_product_attention(queries, keys, values, mask):
  product = tf.matmul(queries, keys, transpose_b=True)
  keys_dim = tf.cast(tf.shape(keys)[-1], tf.float32)
  scaled_product = product / tf.math.sqrt(keys_dim)

  if mask is not None:
    scaled_product += (mask * -1e9) # 0.0000000001

  attention = tf.matmul(tf.nn.softmax(scaled_product, axis=-1), values)
  return attention

### Multi-head attention sublayer

In [ ]:
class MultiHeadAttention(layers.Layer):

    def __init__(self, nb_proj):
      super(MultiHeadAttention, self).__init__()
      self.nb_proj = nb_proj

    def build(self, input_shape):
      self.d_model = input_shape[-1]
      assert self.d_model % self.nb_proj == 0

      self.d_proj = self.d_model // self.nb_proj

      self.query_lin = layers.Dense(units = self.d_model)
      self.key_lin = layers.Dense(units = self.d_model)
      self.value_lin = layers.Dense(units = self.d_model)

      self.final_lin = layers.Dense(units = self.d_model)

    def split_proj(self, inputs, batch_size): # inputs: (batch_size, seq_lenght, d_model)
      shape = (batch_size, -1, self.nb_proj, self.d_proj)
      splited_inputs = tf.reshape(inputs, shape = shape) # (batch_size, seq_lenght, nb_proj, d_proj)
      return tf.transpose(splited_inputs, perm=[0, 2, 1, 3]) # (batch_size, nb_proj, seq_lenght, d_proj)

    def call(self, queries, keys, values, mask):
      batch_size = tf.shape(queries)[0]

      queries = self.query_lin(queries)
      keys = self.key_lin(keys)
      values = self.value_lin(values)

      queries = self.split_proj(queries, batch_size)
      keys = self.split_proj(keys, batch_size)
      values = self.split_proj(values, batch_size)

      attention = scaled_dot_product_attention(queries, keys, values, mask)

      attention = tf.transpose(attention, perm=[0, 2, 1, 3])

      concat_attention = tf.reshape(attention, shape=(batch_size, -1, self.d_model))

      outputs = self.final_lin(concat_attention)

      return outputs

## Encoder

In [ ]:
class EncoderLayer(layers.Layer):

    def __init__(self, FFN_units, nb_proj, dropout_rate):
      super(EncoderLayer, self).__init__()
      self.FFN_units = FFN_units
      self.nb_proj = nb_proj
      self.dropout_rate = dropout_rate

    def build(self, input_shape):
      self.d_model = input_shape[-1]

      self.multi_head_attention = MultiHeadAttention(self.nb_proj)
      self.dropout_1 = layers.Dropout(rate=self.dropout_rate)
      self.norm_1 = layers.LayerNormalization(epsilon=1e-6) # 0.0000001

      self.dense_1 = layers.Dense(units=self.FFN_units, activation='relu')
      self.dense_2 = layers.Dense(units=self.d_model, activation='relu')
      self.dropout_2 = layers.Dropout(rate=self.dropout_rate)

      self.norm_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, mask, training):
      attention = self.multi_head_attention(inputs, inputs, inputs, mask)
      attention = self.dropout_1(attention, training = training)
      attention = self.norm_1(attention + inputs)

      outputs = self.dense_1(attention)
      outputs = self.dense_2(outputs)
      outputs = self.dropout_2(outputs, training=training)
      outputs = self.norm_2(outputs + attention)

      return outputs

In [ ]:
class Encoder(layers.Layer):

    def __init__(self,
                 nb_layers,
                 FFN_units,
                 nb_proj,
                 dropout_rate,
                 vocab_size,
                 d_model,
                 name="encoder"):
      super(Encoder, self).__init__(name=name)
      self.nb_layers = nb_layers
      self.d_model = d_model

      self.embedding = layers.Embedding(vocab_size, d_model)
      self.pos_encoding = PositionalEncoding()
      self.dropout = layers.Dropout(rate=dropout_rate)
      self.enc_layers = [EncoderLayer(FFN_units, nb_proj, dropout_rate) for _ in range(nb_layers)]


    def call(self, inputs, mask, training):
      outputs = self.embedding(inputs)
      outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
      outputs = self.pos_encoding(outputs)
      outputs = self.dropout(outputs, training)

      for i in range(self.nb_layers):
        outputs = self.enc_layers[i](outputs, mask, training)

      return outputs

## Decoder

In [ ]:
class DecoderLayer(layers.Layer):

    def __init__(self, FFN_units, nb_proj, dropout_rate):
      super(DecoderLayer, self).__init__()
      self.FFN_units = FFN_units
      self.nb_proj = nb_proj
      self.dropout_rate = dropout_rate

    def build(self, input_shape):
      self.d_model = input_shape[-1]

      self.multi_head_attention_1 = MultiHeadAttention(self.nb_proj)
      self.dropout_1 = layers.Dropout(rate=self.dropout_rate)
      self.norm_1 = layers.LayerNormalization(epsilon=1e-6)

      self.multi_head_attention_2 = MultiHeadAttention(self.nb_proj)
      self.dropout_2 = layers.Dropout(rate=self.dropout_rate)
      self.norm_2 = layers.LayerNormalization(epsilon=1e-6)

      self.dense_1 = layers.Dense(units = self.FFN_units, activation='relu')
      self.dense_2 = layers.Dense(units = self.d_model, activation='relu')
      self.dropout_3 = layers.Dropout(rate=self.dropout_rate)
      self.norm_3 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, enc_outputs, mask_1, mask_2, training):
      attention = self.multi_head_attention_1(inputs, inputs, inputs, mask_1)
      attention = self.dropout_1(attention, training)
      attention = self.norm_1(attention + inputs)

      attention_2 = self.multi_head_attention_2(attention, enc_outputs, enc_outputs, mask_2)
      attention_2 = self.dropout_2(attention_2, training)
      attention_2 = self.norm_2(attention_2 + attention)

      outputs = self.dense_1(attention_2)
      outputs = self.dense_2(outputs)
      outputs = self.dropout_3(outputs, training)
      outputs = self.norm_3(outputs + attention_2)

      return outputs

In [ ]:
class Decoder(layers.Layer):

    def __init__(self,
                 nb_layers,
                 FFN_units,
                 nb_proj,
                 dropout_rate,
                 vocab_size,
                 d_model,
                 name="decoder"):
      super(Decoder, self).__init__(name=name)
      self.d_model = d_model
      self.nb_layers = nb_layers

      self.embedding = layers.Embedding(vocab_size, d_model)
      self.pos_encoding = PositionalEncoding()
      self.dropout = layers.Dropout(rate=dropout_rate)

      self.dec_layers = [DecoderLayer(FFN_units, nb_proj, dropout_rate) for i in range(nb_layers)]

    def call(self, inputs, enc_outputs, mask_1, mask_2, training):
      outputs = self.embedding(inputs)
      outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
      outputs = self.pos_encoding(outputs)
      outputs = self.dropout(outputs, training)

      for i in range(self.nb_layers):
        outputs = self.dec_layers[i](outputs, enc_outputs, mask_1, mask_2, training)

      return outputs

## Transformer

- Matriz triangular: https://mundoeducacao.bol.uol.com.br/matematica/matriz-triangular.htm

In [ ]:
class Transformer(tf.keras.Model):

    def __init__(self,
                 vocab_size_enc,
                 vocab_size_dec,
                 d_model,
                 nb_layers,
                 FFN_units,
                 nb_proj,
                 dropout_rate,
                 name="transformer"):
        super(Transformer, self).__init__(name=name)

        self.encoder = Encoder(nb_layers, FFN_units, nb_proj, dropout_rate,
                               vocab_size_enc, d_model)
        self.decoder = Decoder(nb_layers, FFN_units, nb_proj, dropout_rate,
                               vocab_size_dec, d_model)
        self.last_linear = layers.Dense(units=vocab_size_dec, name='lin_output')

    def create_padding_mask(self, seq): # (batch_size, seq_length) -> (batch_size, nb_proj, seq_lenght, d_proj)
      mask = tf.cast(tf.math.equal(seq, 0), tf.float32)
      return mask[:, tf.newaxis, tf.newaxis, :]

    def create_look_ahead_mask(self, seq):
      seq_len = tf.shape(seq)[1]
      look_ahed_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
      return look_ahed_mask

    def call(self, enc_inputs, dec_inputs, training):
      enc_mask = self.create_padding_mask(enc_inputs)
      dec_mask_1 = tf.maximum(self.create_padding_mask(dec_inputs), self.create_look_ahead_mask(dec_inputs))
      dec_mask_2 = self.create_padding_mask(enc_inputs)

      enc_outputs = self.encoder(enc_inputs, enc_mask, training)
      dec_outputs = self.decoder(dec_inputs, enc_outputs, dec_mask_1, dec_mask_2, training)

      outputs = self.last_linear(dec_outputs)

      return outputs

## Código somente para testes

In [ ]:
# Código somente para testes

def create_padding_mask(seq): # (batch_size, seq_length) -> (batch_size, nb_proj, seq_lenght, d_proj)
  mask = tf.cast(tf.math.equal(seq, 0), tf.float32)
  return mask[:, tf.newaxis, tf.newaxis, :]

def create_look_ahead_mask(seq):
  seq_len = tf.shape(seq)[1]
  look_ahed_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  return look_ahed_mask

In [ ]:
seq = tf.cast([[837, 836, 0, 273, 8, 0, 0, 0]], tf.int32)

In [ ]:
create_padding_mask(seq)

<tf.Tensor: shape=(1, 1, 1, 8), dtype=float32, numpy=array([[[[0., 0., 1., 0., 0., 1., 1., 1.]]]], dtype=float32)>

In [ ]:
create_look_ahead_mask(seq)

<tf.Tensor: shape=(8, 8), dtype=float32, numpy=
array([[0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [ ]:
tf.maximum(create_padding_mask(seq), create_look_ahead_mask(seq))

<tf.Tensor: shape=(1, 1, 8, 8), dtype=float32, numpy=
array([[[[0., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1., 1., 1., 1.],
         [0., 0., 1., 0., 1., 1., 1., 1.],
         [0., 0., 1., 0., 0., 1., 1., 1.],
         [0., 0., 1., 0., 0., 1., 1., 1.],
         [0., 0., 1., 0., 0., 1., 1., 1.],
         [0., 0., 1., 0., 0., 1., 1., 1.]]]], dtype=float32)>

# Treinamento

In [ ]:
tf.keras.backend.clear_session()

d_model = 128 # 512
nb_layers = 4 # 6
ffn_units = 512 # 2048
nb_proj = 8 # 8
dropout_rate = 0.1 # 0.1

In [ ]:
transformer = Transformer(vocab_size_enc=vocab_size_en,
                          vocab_size_dec=vocab_size_pt,
                          d_model=d_model,
                          nb_layers=nb_layers,
                          FFN_units=ffn_units,
                          nb_proj=nb_proj,
                          dropout_rate=dropout_rate)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction='none')

In [ ]:
def loss_function(target, pred):
  mask = tf.math.logical_not(tf.math.equal(target, 0))
  loss_ = loss_object(target, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = tf.cast(d_model, tf.float32)
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = CustomSchedule(d_model)

In [ ]:
# ATUALIZAÇÃO MAR-2024
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [ ]:
checkpoint_path = "/content/drive/MyDrive/tradutor"
ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print('Latest checkpoint restored')

In [ ]:
epochs = 10
for epoch in range(epochs):
  print('Start or epoch {}'.format(epoch + 1))
  start = time.time()

  train_loss.reset_states()
  train_accuracy.reset_states()

  for (batch, (enc_inputs, targets)) in enumerate(dataset):
    dec_inputs = targets[:, :-1]
    dec_outputs_real = targets[:, 1:]
    with tf.GradientTape() as tape:
      predictions = transformer(enc_inputs, dec_inputs, True)
      loss = loss_function(dec_outputs_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    train_accuracy(dec_outputs_real, predictions)

    if batch % 50 == 0:
      print('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch+1, batch, train_loss.result(), train_accuracy.result()))

  ckpt_save_path = ckpt_manager.save()
  print('Saving checkpoint for epoch {} at {}'.format(epoch + 1, ckpt_save_path))
  print('Time taken for 1 epoch {} secs\n'.format(time.time() - start))

# Avaliação

In [ ]:
text = 'you are smart'
text = [vocab_size_en - 2] + tokenizer_en.encode(text) + [vocab_size_en - 1]
text

[8191, 55, 17, 2202, 4099, 8192]

In [ ]:
text = tf.expand_dims(text, axis=0)
text.shape

TensorShape([1, 6])

In [ ]:
output = tf.expand_dims([vocab_size_pt - 2], axis = 0)
output.shape

TensorShape([1, 1])

In [ ]:
def evaluate(inp_sentence):
  inp_sentence = [vocab_size_en - 2] + tokenizer_en.encode(inp_sentence) + [vocab_size_en - 1]
  enc_input = tf.expand_dims(inp_sentence, axis=0)

  output = tf.expand_dims([vocab_size_pt - 2], axis = 0)

  # i am -> am happy

  for _ in range(max_length):
    # (1, seq_length, vocab_size)
    predictions = transformer(enc_input, output, False)
    prediction = predictions[:, -1:, :]

    predicted_id = tf.cast(tf.argmax(prediction, axis=-1), tf.int32)

    if predicted_id == vocab_size_pt - 1:
      return tf.squeeze(output, axis=0)

    output = tf.concat([output, predicted_id], axis=1)

  return tf.squeeze(output, axis = 0)

In [ ]:
def translate(sentence):
  output = evaluate(sentence).numpy()

  predicted_sentence = tokenizer_pt.decode([i for i in output if i < vocab_size_pt - 2])

  print('Input: {}'.format(sentence))
  print('Predicted translation: {}'.format(predicted_sentence))


In [ ]:
translate("this is a really powerful tool")

Input: this is a really powerful tool
Predicted translation: Também isto é um instrumento podíssimo.


Melhorias

- Utilizar a base de dados completa
- Aumentar o tamanho da frase
- Mudar os parâmetros